In [1]:
import os
os.chdir("..")
from app.log import logging_config
from os.path import dirname, join
from dotenv import load_dotenv
import logging
from app.util.message import starting_message
from app import Polygon
from app import Twitter
import pandas as pd
import time
import datetime 

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

c:\Users\andre\Anaconda3\envs\DS4A_PF\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#FinancialBERT model
financialbert_model = BertForSequenceClassification.from_pretrained(".\\sentiment_models\\financial_bert\\financialbert_model")
financialbert_tokenizer = BertTokenizer.from_pretrained(".\\sentiment_models\\financial_bert\\financialbert_tokenizer")
nlp_financial = pipeline("sentiment-analysis", model=financialbert_model, tokenizer=financialbert_tokenizer)

#FinBERT model
finbert_model = BertForSequenceClassification.from_pretrained(".\\sentiment_models\\finbert\\finbert_model")
finbert_tokenizer = BertTokenizer.from_pretrained(".\\sentiment_models\\finbert\\finbert_tokenizer")
nlp_fin = pipeline("sentiment-analysis", model=finbert_model, tokenizer=finbert_tokenizer)

In [3]:
#Top 50 cripto currencies.
top_crypto = pd.read_csv("./presel_coins.csv")
top_crypto['days_history'] = top_crypto['days_history'].str[0:5].astype('int')
top_crypto = top_crypto[['rank', 'symbol', 'days_history']].sort_values(by = ['rank'])
top_crypto['init_date'] = pd.to_datetime("2022-6-24") - pd.to_timedelta(top_crypto['days_history'], unit = 'd')
top_crypto = top_crypto.reset_index(drop = True)
top_crypto

,rank,symbol,days_history,init_date
0,1,BTC,3339,2013-05-03
1,2,ETH,2508,2015-08-12
2,3,USDT,2671,2015-03-02
3,8,XRP,3241,2013-08-09
4,10,DOGE,3108,2013-12-20
5,18,LTC,3339,2013-05-03
6,24,XLM,2875,2014-08-10
7,29,XMR,2951,2014-05-26
8,70,DASH,3047,2014-02-19
9,83,XEM,2636,2015-04-06


In [4]:
load_dotenv()
from os import environ
print(environ.get("POLYGON_KEY"))
logging_config.init_logging()
__LOG = logging.getLogger(__name__)
__LOG.info('...... Initialization Completed  ......')
__LOG.info(starting_message())

PWRjSfmBc9xYnPZMQhA9ukDOuOIId90D


In [5]:
polygon=Polygon(environ.get("POLYGON_KEY"))

In [6]:
cryptonews_df_list = []
for j in range(0, top_crypto.shape[0]):
    if (j + 1) %5 == 0: time.sleep(65) #5 requests per minute
    try:
        status, polygon_data = polygon.get_news_ticker('', top_crypto['symbol'].loc[j])
        if status:
            polygon_news_ticker = pd.DataFrame.from_dict(polygon_data['results'])
        for i in polygon_news_ticker.index:
            polygon_news_ticker_pbs = polygon_news_ticker['publisher'][i]['name']
            polygon_news_ticker['publisher'][i] = polygon_news_ticker_pbs
        print(status, top_crypto['symbol'].loc[j])
        print(polygon_news_ticker['published_utc'].min(), polygon_news_ticker['published_utc'].max(), polygon_news_ticker.shape[0])
        cryptonews_df_list.append(polygon_news_ticker)
    except:
        pass


True BTC
2021-03-06T12:10:00Z 2021-06-15T22:10:55Z 80
True ETH
2021-04-25T09:30:00Z 2021-08-16T14:39:00Z 244
True USDT
True XRP
True DOGE
True LTC
2021-04-25T13:42:00Z 2022-06-27T13:16:00Z 110
True XLM
True XMR
True DASH
2021-04-23T11:15:00Z 2022-06-27T19:44:00Z 469
True XEM
True SC
2021-04-06T14:15:00Z 2022-02-01T11:43:54Z 63
True DGB
True SYS
True REP
True MAID
True XWC
True XVG
True MONA
True LOG
True GRS
True BTS
True COVAL
True NMC
True RDD
True BCN
True IFC
True VTC
2021-06-17T09:30:00Z 2022-06-25T06:15:00Z 8
True NXS
True PPC
2020-06-29T13:15:00Z 2022-06-24T20:08:09Z 236
True XCP
True AGRS
True DMD
True PAC
2021-05-15T16:08:00Z 2022-05-05T23:15:20Z 9
True NAV
2021-05-10T16:13:44Z 2021-06-28T12:50:07Z 15
True BITCNY
True GAME
2021-07-08T10:59:42Z 2021-11-29T14:55:39Z 3
True CARBON
True NXT
True FCT
2022-03-17T04:51:12Z 2022-03-17T04:51:12Z 1
True SIB
True EMC2
True CLOAK
True OMNI
True DIME
True GRC
2021-04-26T08:24:04Z 2021-12-05T05:58:49Z 4
True TX
2020-10-30T14:15:00Z 2022-06-

In [7]:
#Unique days retrieved from Polygon 
for df in  cryptonews_df_list: print(df['published_utc'].str[0:9].nunique())

11
16
41
54
25
8
37
8
5
2
1
4
41
5


In [15]:

twt = Twitter()

TypeError: Twitter.__init__() takes 1 positional argument but 2 were given

In [10]:

coins_df = pd.read_csv('presel_coins.csv')
tweets_dfs = []
for ticker in coins_df['symbol'].unique():
    q = coins_df[coins_df['symbol']==ticker]['twt_query'].values[0]
    ticker_tweets = twt.get_tweets_df(query=q, limit=3)
    if ticker_tweets.size > 0:
        ticker_tweets['Ticker'] = ticker
        tweets_dfs.append(ticker_tweets)

print(pd.concat(tweets_dfs, ignore_index=True))

NameError: name 'twt' is not defined